# Chapter 4

## Query Enhancement

In [1]:
%load_ext autoreload
%autoreload 2

import json
import pathlib
import asyncio

import cohere
import weave

import wandb

In [2]:
WANDB_ENTITY = "rag-course"
WANDB_PROJECT = "dev"

wandb.require("core")

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    group="Chapter 4",
)


weave_client = weave.init(f"{WANDB_ENTITY}/{WANDB_PROJECT}")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ayut (rag-course). Use `wandb login --relogin` to force relogin


weave version 0.50.10 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: ayut.
View Weave data at https://wandb.ai/rag-course/dev/weave


In [3]:
# Reload the data from Chapter 3
chunked_artifact = run.use_artifact(
    f"{WANDB_ENTITY}/{WANDB_PROJECT}/chunked_data:latest", type="dataset"
)
artifact_dir = chunked_artifact.download()
chunked_data_file = pathlib.Path(f"{artifact_dir}/documents.jsonl")
chunked_data = list(map(json.loads, chunked_data_file.read_text().splitlines()))
chunked_data[:2]

2024/07/19 18:43:07 [DEBUG] GET https://storage.googleapis.com/wandb-production.appspot.com/rag-course/dev/j8uh2i2o/artifact/961260984/wandb_manifest.json?Expires=1721398387&GoogleAccessId=gorilla-files-url-signer-man%40wandb-production.iam.gserviceaccount.com&Signature=OItAjeFlEwOfK5aI3%2FMMh7AoUgiyHE9msVoPIhoEd5MPhVJ1BRfF%2Fkh%2Bp0Pn%2BqT0xKzEws%2ByIXx7VqDyUirEHQ8hGlDjwhaVd1DZA5VmaJ3OZW%2FQGgcPPJiHMLqK8z89v0ydhjTKw3k0%2FxELXjmxBHq1an7cVZwFQ9EcZJywjjySUUiFiBG6psoAwRZjbG0cKD6tCm6%2FbNdzU%2FW7i60dwW11QCuh9Bmrj9ZziuJnjeje1EFUetkBdd%2BjtJjntgi9icrbvACGfykmCC2fokXhJh9umSVsJbPTC8q0FB34gJy0JLwAGBiG3owBzAdKiqkcjpimweknOL9%2Br3SHfwa%2FQg%3D%3D


[{'cleaned_content': 'Anonymous Mode Are you publishing code that you want anyone to be able to run easily? Use Anonymous Mode to let someone run your code, see a W&B dashboard, and visualize results without needing to create a W&B account first. Allow results to be logged in Anonymous Mode with wandb.init(anonymous="allow") :::info Publishing a paper? Please cite W&B, and if you have questions about how to make your code accessible while using W&B, reach out to us at support@wandb.com.\n::: How does someone without an account see results? If someone runs your script and you have to set anonymous="allow":  Auto-create temporary account: W&B checks for an account that\'s already signed in. If there\'s no account, we automatically create a new anonymous account and save that API key for the session. Log results quickly: The user can run and re-run the script, and automatically see results show up in the W&B dashboard UI.\nThese unclaimed anonymous runs will be available for 7 days. Claim

In [4]:
# The query enhancer will be used to identify the language, the intent of the query and generate search queries for retrieval
# this additional information will be used to inform the response generator and improve the retrieval process


In [7]:
from scripts.query_enhancer import QueryEnhancer
from scripts.utils import display_source

query_enhancer = QueryEnhancer()
display_source(query_enhancer)

Unable to get source code for QueryEnhancer(name=None, description=None). It might be a built-in or compiled object.


In [8]:
response = await query_enhancer.predict("How do I log images in lightning with wandb?")

🍩 https://wandb.ai/rag-course/dev/r/call/f036a49b-c2d2-4d0f-b4b5-f6932f18d1a3


In [9]:
# the retriever remains the same

from scripts.retriever import BM25Retriever
retriever = BM25Retriever()
retriever.index_data(chunked_data)

/Users/ayushthakur/miniconda3/envs/rag-irl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# we need to improve the Response generator to incorporate the new information from the query enhancer
# such as adding in the language identification and the intent action to the prompt

from scripts.response_generator import QueryEnhanedResponseGenerator
display_source(QueryEnhanedResponseGenerator)


   1 class QueryEnhanedResponseGenerator(weave.Model):                                                             
   2     model: str                                                                                                
   3     prompt: str                                                                                               
   4     client: cohere.AsyncClient = None                                                                         
   5                                                                                                               
   6     def __init__(self, **kwargs):                                                                             
   7         super().__init__(**kwargs)                                                                            
   8         self.client = cohere.AsyncClient(api_key=os.environ["CO_API_KEY"])                                    
   9                                                                                                               
  10     @weave.op()                                                                                               
  11     def generate_context(self, context: List[Dict[str, any]]) -> List[Dict[str, any]]:                        
  12         return [{"source": item["source"], "text": item["text"]} for item in context]                         
  13                                                                                                               
  14     @weave.op()                                                                                               
  15     async def generate_response(                                                                              
  16         self,                                                                                                 
  17         query: str,                                                                                           
  18         context: List[Dict[str, any]],                                                                        
  19         language: str,                                                                                        
  20         intents: List[str],                                                                                   
  21     ) -> str:                                                                                                 
  22         contexts = self.generate_context(context)                                                             
  23         response = await self.client.chat(                                                                    
  24             preamble=self.prompt.format(language=language, intents=intents),                                  
  25             message=query,                                                                                    
  26             model=self.model,                                                                                 
  27             documents=contexts,                                                                               
  [38;2;101;102;96

In [11]:
from scripts.rag_pipeline import QueryEnhancedRAGPipeline
display_source(QueryEnhancedRAGPipeline)

   1 class QueryEnhancedRAGPipeline(weave.Model):                                                                  
   2     query_enhancer: weave.Model = None                                                                        
   3     retriever: weave.Model = None                                                                             
   4     response_generator: weave.Model = None                                                                    
   5     top_k: int = 5                                                                                            
   6                                                                                                               
   7     @weave.op()                                                                                               
   8     async def predict(self, query: str):                                                                      
   9         # enhance the query                                                                                   
  10         enhanced_query = await self.query_enhancer.predict(query)                                             
  11         user_query = enhanced_query["query"]                                                                  
  12                                                                                                               
  13         avoid_intents = [                                                                                     
  14             "unrelated",                                                                                      
  15             "needs_more_info",                                                                                
  16             "opinion_request",                                                                                
  17             "nefarious_query",                                                                                
  18             "other",                                                                                          
  19         ]                                                                                                     
  20                                                                                                               
  21         avoid_retrieval = False                                                                               
  22                                                                                                               
  23         intents = enhanced_query["intents"]                                                                   
  24         for intent in intents:                                                                                
  25             if intent["intent"] in avoid_intents:                                                             
  26                 avoid_retrieval = True                                                                        
  27                 break                                                                                         
  28                                                                                                               
  29         language = enhanced_query["language"]                                                                 
  30                                                                                                               
  31         # run retrieval for multiple queries                                                                  
  32         contexts = []                                                                                         
  33         if not avoid_retrieval:                                                                               
  34             retriever_queries = enhanced_query["search_queries"]                                              
  35             contexts.append(self.retriever.predict(

In [12]:
# lets add the new prompt
QUERY_ENHANCED_PROMPT = open("prompts/query_enhanced_system.txt").read()

response_generator = QueryEnhanedResponseGenerator(
    model="command-r-plus", prompt=QUERY_ENHANCED_PROMPT, client=cohere.AsyncClient()
)

In [13]:
# similarily we will also improve the rag pipeline to first call the query enhancer and the use the information
# to include logic to aviod retrieval when the query is unrelated, needs more info, etc.
# extract the language and intent actions so we can instruct the response generator to use them

rag_pipeline = QueryEnhancedRAGPipeline(
    query_enhancer=query_enhancer,
    retriever=retriever,
    response_generator=response_generator,
)

response = await rag_pipeline.predict("How do I log images in lightning with wandb?")
response

🍩 https://wandb.ai/rag-course/dev/r/call/07daf50a-ccac-49c4-b799-d903fc2e3c18


'The core integration of W&B with Lightning is based on the Lightning loggers API, which lets you write most of your logging code in a framework-agnostic way. Loggers are passed to the Lightning Trainer and are triggered based on that API\'s rich hook-and-callback system.\n\nTo log images, you can use the wandb.Image() function. Here is an example code snippet:\n\n```python\nwandb.log({"example": wandb.Image("myimage.jpg")})\n```\n\nThis will log an example image that will be displayed in the W&B UI.'

## Evaluate and Compare

In [14]:
eval_dataset = weave.ref(
    "weave:///rag-course/dev/object/Dataset:9O0EmmPINmYjgbXW3kucVrDxlTUQJQs0fVZYJj2mtOk"
).get()

In [15]:
from scripts.response_metrics import ALL_METRICS as RESPONSE_METRICS

response_evaluations = weave.Evaluation(
    name="Response_Evaluation",
    dataset=eval_dataset,
    scorers=RESPONSE_METRICS,
    preprocess_model_input=lambda x: {"query": x["question"]},
)
query_enhanced_response_scores = asyncio.run(
    response_evaluations.evaluate(rag_pipeline)
)

Evaluated 1 of 50 examples

Evaluated 2 of 50 examples

Evaluated 3 of 50 examples

Evaluated 4 of 50 examples

Evaluated 5 of 50 examples

...

Evaluated 48 of 50 examples

Evaluated 49 of 50 examples

Evaluated 50 of 50 examples

Evaluation summary
{
    'compute_diff': {'mean': 0.03041334882732394},
    'compute_levenshtein': {'mean': 0.39822341192820276},
    'compute_rouge': {'mean': 0.22057337543398303},
    'compute_bleu': {'mean': 0.05010284259894616},
    'llm_response_scorer': {'score': {'mean': 0.56}, 'correct': {'true_count': 1, 'true_fraction': 0.02}},
    'model_latency': {'mean': 22.344214463233946}
}

🍩 https://wandb.ai/rag-course/dev/r/call/f94d4c7a-a986-4a76-a728-cc5646255550


Exercise 